In [1]:
def ConvertLong(x):
    """
    Pour passer de Cm^-1 à nanomètres ou l'inverse.
    """
    return 10**7/x

def Convert2deg(hours_or_degs, min, sec, mode="hms", decimals=4):
    """
    Convertit les heures, minutes et secondes ("hms") ou degrés, minutes, secondes ("dms") en degrés avec précision donnée par "decimals".
    """
    if mode=="hms":
        return np.round((hours_or_degs + min/60 + sec/3600) * 360/24, decimals=decimals)
    elif mode=="dms":
        return np.round(hours_or_degs + min/60 + sec/3600, decimals=decimals)
    else:
        print("Mode incorrect, 'hms' ou 'dms' sont acceptés.")

def Convert_from_deg(degs, mode="hms", decimals=2):
    """
    Convertit les degrés décimaux en [heures, minutes, secondes] ("hms") ou [degrés, minutes, secondes] ("dms") avec précision sur les secondes
    données par "decimals".
    """
    if mode=="hms":
        hours_dec = degs*24/360
        hours = np.floor(hours_dec)
        mins_dec = (hours_dec - hours)*60
        mins = np.floor(mins_dec)
        secs = np.round((mins_dec - mins)*60 ,decimals=decimals)
        return [hours, mins, secs]
    elif mode=="dms":
        degs_round = np.floor(degs)
        mins_dec = (degs - degs_round)*60
        mins = np.floor(mins_dec)
        secs = np.round((mins_dec - mins)*60 ,decimals=decimals)
        return [degs_round, mins, secs]
    else:
        print("Mode incorrect, 'hms' ou 'dms' sont acceptés.")

def Find_lines(ax1):
    """
    Choisit les raies que l'on veut afficher en particulier, en prenant uniquement celles qui se trouvent
    entre le min et le max des longueurs d'ondes de ax1.

    Retourne un tableau avec le nom des raies choisies et un tableau avec les valeurs de longueurs d'ondes.
    """
    xdata = ax1.lines[0].get_xdata()                       #data en x en cm^-1
    xi, xf = np.min(xdata), np.max(xdata)                  #min et max des nbr d'ondes
    lf, li = ConvertLong(xi), ConvertLong(xf)    #min et max des long d'ondes
    
    lines = np.array(["H\u03b1", "[SII] ", "[NII]", "[NII]", "HeI", "[OIII]", "[OIII]", "H\u03b2"])       #raies que l'on veut afficher            
    lines_l = np.array([656.3, 672.2, 658.3, 654.8, 667.8, 495.9, 500.7, 486.1])         #valeurs correspondantes en nm
    
    #Pour chaque raie, on regarde si la val en nm se trouve dans l'intervalle des données. Si oui, on remplit les arrays chosen_lines
    chosen_lines, chosen_lines_l = np.array([]), np.array([])
    for raie_l in lines_l:
        raie = lines[np.where(lines_l == raie_l)[0]]
        if ((raie_l > li) & (raie_l < lf)):
            chosen_lines_l = np.append(chosen_lines_l, raie_l)
            chosen_lines = np.append(chosen_lines, raie)

    return chosen_lines, chosen_lines_l

def lines_ticks(ax1):
    """
    Utilise Find_lines pour ajouter les ticks et labels des raies 
    qu'on veut afficher en particulier, en ticks minors, sur un axe
    en haut de la figure en longueurs d'ondes.
    """
    chosen_lines, chosen_lines_l = Find_lines(ax1)

    #Ajoute les raies choisies en ticks minors, avec paramètres souhaités
    secax = ax1.secondary_xaxis('top', functions=(ConvertLong, ConvertLong))
    secax.set_xticks(chosen_lines_l, minor=True)
    secax.set_xticklabels(chosen_lines, minor=True)
    secax.tick_params(axis='x', which='minor', color='black', labelcolor='navy', direction='in', pad=-20, labelrotation=90, labelsize=9)
    
    #Pour éviter les messages warnings inutiles :
    import warnings
    warnings.filterwarnings("ignore", message="divide by zero encountered in divide")

    return secax

def limSN1():
    """
    Pose les limites du filtre SN1
    """
    xf = ConvertLong(358.)
    xi = ConvertLong(391.)
    plt.xlim((xi, xf))

def limSN2():
    """
    Pose les limites du filtre SN2
    """
    xf = ConvertLong(474.)
    xi = ConvertLong(522.)
    plt.xlim((xi, xf))

def limSN3():
    """
    Pose les limites du filtre SN3
    """
    xf = ConvertLong(646.)
    xi = ConvertLong(689.)
    plt.xlim((xi, xf))

def xlim2(li, lf):
    """
    Pose les limites voulues en longueurs d'ondes (nm)
    """
    xi = ConvertLong(lf)
    xf = ConvertLong(li)
    plt.xlim((xi, xf))

def draw_region(ax, header, reg_name, set_lim = None, offset=None):
    """
    Dessine la région choisie (fichier .reg) sur la figure. Retourne la région pour
    extraire les spectres etc.
    """
    reg = pyregion.open(reg_name).as_imagecoord(header)
    patch_list, artist_list = reg.get_mpl_patches_texts()
    for p in patch_list:
        ax.add_patch(p)
    for t in artist_list:
        ax.add_artist(t)

    if set_lim != None:
        offset_arcsec = offset
        offset_degree = offset_arcsec/3600
        reg_sky_coord = pyregion.open(reg_name)[0].coord_list
        if set_lim == "box":
            xlim_arr = np.array([reg_sky_coord[0] - (reg_sky_coord[2]/np.cos(reg_sky_coord[1]))/2 - offset_degree/np.cos(reg_sky_coord[1]) , reg_sky_coord[0] + (reg_sky_coord[2]/np.cos(reg_sky_coord[1]))/2 + offset_degree/np.cos(reg_sky_coord[1])])
            ylim_arr = np.array([reg_sky_coord[1] - reg_sky_coord[3]/2 - offset_degree , reg_sky_coord[1] + reg_sky_coord[3]/2 + offset_degree])
        elif set_lim == "circle":
            xlim_arr = np.array([reg_sky_coord[0] - (reg_sky_coord[2]/np.cos(reg_sky_coord[1])) - offset_degree/np.cos(reg_sky_coord[1]) , reg_sky_coord[0] + reg_sky_coord[2]/np.cos(reg_sky_coord[1]) + offset_degree/np.cos(reg_sky_coord[1])])
            ylim_arr = np.array([reg_sky_coord[1] - reg_sky_coord[2] - offset_degree , reg_sky_coord[1] + reg_sky_coord[2] + offset_degree])

        lim_coords = SkyCoord(xlim_arr , ylim_arr, unit="deg")
        lim_xpixels, lim_ypixels = WCS(header).world_to_pixel(lim_coords)
        plt.xlim(lim_xpixels)
        plt.ylim(lim_ypixels)

    return reg

def draw_map(path, title=None, cbar="viridis", format_x="d.dd", format_y="d.dd", return_header=False, perc=99.5):
    """
    Dessine la carte donnée en .fits avec chemin "path", sur la figure "fig" avec les axes "ax" en unités physiques.
    Le style de colorbar peut être changé (voir https://matplotlib.org/stable/users/explain/colors/colormaps.html).
    Retourne le header de la map en plus si "return_header" est "True".
    """
    map, header_map = orb.utils.io.read_fits(path, return_header=True)
    # header_map["CTYPE1"] = 'RA---TAN-SIP'
    # header_map["CTYPE2"] = 'DEC--TAN-SIP'
    wcs_map = WCS(header_map)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection=wcs_map)
    ax.coords[0].set_major_formatter(format_x)
    ax.coords[1].set_major_formatter(format_y)
    plt.imshow(map.T, origin='lower',
              vmin=np.nanpercentile(map, 100-perc),
              vmax=np.nanpercentile(map, perc),
              cmap=cbar)
    plt.colorbar()
    if title!=None : plt.title(title)
    
    if return_header == False: return fig, ax
    elif return_header==True: return fig, ax, header_map

def draw_WR(header, input_wcs=None, markersize=7, color="yellow"):
    """
    Trace les étoiles WR contenues dans l'image dont le header est donné en argument. Fonctionne uniquement pour M31 et retourne les noms des WRs.
    Source : Neugent et al. 2012 (https://iopscience.iop.org/article/10.1088/0004-637X/759/1/11), table 5.

    Nécessite le header de l'image et le wcs. Si "input_wcs" n'est pas donné, le wcs est trouvé avec fonction astropy.wcs sur le header.
    (Distinction nécessaire car WCS(header) ne fonctionne pas avec le header du cube en entier ou du deep frame mais bien avec les headers d'images .fits).
    """
    names, hours, xmins, xsecs, degs, ymins, ysecs, hours_dec, degs_dec = np.loadtxt("List_WR_M31_coord.txt", unpack=True, delimiter=";", dtype=str)
    hours = hours.astype(float) ; xmins = xmins.astype(float) ; xsecs = xsecs.astype(float) ; hours_dec = hours_dec.astype(float)
    degs = degs.astype(float) ; ymins = ymins.astype(float) ; ysecs = ysecs.astype(float) ; degs_dec = degs_dec.astype(float)

    if input_wcs==None : wcs_map = WCS(header)
    elif input_wcs!=None: wcs_map = input_wcs

    #Limites du cube en degrés décimaux trouvées à partir des max et mins des 4 coins (valeurs extrêmes viennent forcément des coins)
    xpix,ypix=np.meshgrid(np.array([0, header["NAXIS1"]]), np.array([0, header["NAXIS2"]]))    #tab des coordonnées des coins en pixels
    xlim_inf = np.min(wcs_map.pixel_to_world(xpix, ypix[::-1]).ra.degree)
    xlim_sup = np.max(wcs_map.pixel_to_world(xpix, ypix[::-1]).ra.degree)
    ylim_inf = np.min(wcs_map.pixel_to_world(xpix, ypix[::-1]).dec.degree)
    ylim_sup = np.max(wcs_map.pixel_to_world(xpix, ypix[::-1]).dec.degree)

    #Recherche des WR qui se trouvent dans ces limites :
    ind_contained = find_stars(xlim_inf, xlim_sup, ylim_inf, ylim_sup, hours_dec , degs_dec)
    if ind_contained.size == 0: print("Aucune WR dans la zone considérée.")
    WR_coords = SkyCoord(hours_dec[ind_contained] , degs_dec[ind_contained], unit="deg")
    WR_xpixels, WR_ypixels = wcs_map.world_to_pixel(WR_coords)
    plt.plot(WR_xpixels , WR_ypixels , "*", color=color, markersize=markersize)

    return names[ind_contained]

def find_stars(xinf, xsup, yinf, ysup, xcoord, ycoord):
    """
    Trouve les indices des étoiles contenues dans le rectangle délimité par les lims en x (R.A.) et y (DEC)
    à partir de la liste xcoord des R.A. des étoiles en format degrés avec décimales (ex: 1h30min => 22.5°)
    et de la liste ycoord des DEC des étoiles dans le même format.

    xcoord et ycoord doivent être des arrays de même taille.
    """
    ind_stars_contained = np.array([])
    for i in np.arange(xcoord.size):
        x = xcoord[i] ; y = ycoord[i]
        if ((x>xinf) & (x<xsup) & (y>yinf) & (y<ysup)):
            ind_stars_contained = np.append(ind_stars_contained , i)
    return ind_stars_contained.astype(int)

def synchronize_subplots(fig, ax1, ax2):
    def on_xlims_change(event):
    # Get the x-axis limits of the first subplot
        xlim = ax1.get_xlim()
    
        # Apply the same limits to the second subplot
        ax2.set_xlim(xlim)
        fig.canvas.draw_idle()  # Update the figure
    def on_ylims_change(event):
        # Get the x-axis limits of the first subplot
        ylim = ax1.get_ylim()
    
        # Apply the same limits to the second subplot
        ax2.set_ylim(ylim)
        fig.canvas.draw_idle()  # Update the figure
    ax1.callbacks.connect('xlim_changed', on_xlims_change)
    ax1.callbacks.connect('ylim_changed', on_ylims_change)